In [39]:
import numpy as np
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import sys, os
from loguru import logger
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import layers
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.dummy import DummyClassifier
from tensorflow.python.keras.layers import Dense, Flatten, Input, Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.callbacks import EarlyStopping, TensorBoard
from ray.tune.integration.keras import TuneReportCallback
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune import JupyterNotebookReporter
from ray import tune

sys.path.append('..')

from definitions import get_project_root
from src.data.make_dataset import create_train_test_validation
from src.visualization.visualize import plot_results

root = get_project_root()

In [3]:
tf.random.set_seed(42)



In [9]:
## Create train, validation and test sets
x_train, x_valid, x_test, y_train, y_valid, y_test = create_train_test_validation()
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape

2022-02-11 18:09:44.516 | INFO     | src.data.make_dataset:create_train_test_validation:73 - found file labeled_data.csv, procceed with creating train, test and validation sets


((61711, 23), (61711, 1), (13225, 23), (13225, 1), (13224, 23), (13224, 1))

In [40]:
class HyperModel(tf.keras.Model):
    def __init__(self,config):
        super().__init__(config)

        self.dense = Dense(23 , activation=config['activation'])

        self.hidden = []
        for i in range(config["hidden"]):
            self.hidden += [Dense(config["hidden"],activation=config["activation"])]
        
        self.hidden += [Dropout(config["dropout"])]

        self.out = Dense(15, activation="softmax")

    def call(self,x):
        x = self.dense
        
        for layer in self.hidden:
            x = layer(x)
        
        x = self.out(x)
        return x

In [41]:
def train_hyper_model(
        config   
    ): 

    """Using a config file to train the hyper model with the help of the Ray Hypertuner"""
    x_train, x_valid, x_test, y_train, y_valid, y_test = create_train_test_validation() 


    model = HyperModel(config)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer = 'adam',
        metrics = ["accuracy","mape"],
    )
    callbacks = [
        TuneReportCallback(
            {
                "val_loss":"val_loss",
                "accuracy":"accuracy",
                "mape":"mape",
                "val_mape":"val_mape"
            }
        )
    ]

    result = model.fit(
        x_train,
        y_train,
        epochs=config["epochs"],
        validation_data=(x_valid,y_valid),
        callbacks=callbacks,
        verbose=1
    )

    return model

In [42]:
config = {
    "grace_period" : 2, 
    "local_dir" : "logs",
    "epochs" : tune.choice([4,5,6]), 
    "samples" : 15,
    "activation" : "relu",
    "hidden" : tune.randint(1, 5),
    "units" : tune.qrandint(50, 200, 50), 
    "dropout" : tune.uniform(0.05, 0.4)
}

In [43]:
for i in range(config["hidden"]):
    print(i)

TypeError: 'Integer' object cannot be interpreted as an integer

In [44]:
test_model = train_hyper_model(config)

2022-02-11 19:03:49.379 | INFO     | src.data.make_dataset:create_train_test_validation:73 - found file labeled_data.csv, procceed with creating train, test and validation sets


TypeError: 'Integer' object cannot be interpreted as an integer

'penv' is not recognized as an internal or external command,
operable program or batch file.


In [45]:
import ray
ray.init(num_cpus=4,num_gpus=1,ignore_reinit_error=True)
ray.is_initialized()

2022-02-11 19:03:57,367	INFO worker.py:862 -- Calling ray.init() again after it has already been called.


True

In [46]:
sched = AsyncHyperBandScheduler(
    time_attr ="trainingiteration", max_t=200, grace_period=config["grace_period"]
)

reporter = JupyterNotebookReporter(overwrite=True)

analysis = tune.run(
    train_hyper_model,
    config=config,
    name="hypertuner",
    #scheduler=sched,
    metric="val_loss",
    mode="min",
    progress_reporter=reporter,
    local_dir=config["local_dir"],
    stop={"training_iteration":config["epochs"]},
    num_samples=config["samples"])

Trial name,status,loc,dropout,epochs,hidden,stop/training_iteration,units
train_hyper_model_fd4d8_00000,ERROR,,0.339206,6,1,,200
train_hyper_model_fd4d8_00001,ERROR,,0.326508,5,3,,100
train_hyper_model_fd4d8_00002,ERROR,,0.312469,5,2,,150
train_hyper_model_fd4d8_00003,ERROR,,0.0662585,6,4,,100
train_hyper_model_fd4d8_00004,ERROR,,0.104294,5,4,,100
train_hyper_model_fd4d8_00005,ERROR,,0.333241,6,1,,50
train_hyper_model_fd4d8_00006,ERROR,,0.307449,5,3,,150
train_hyper_model_fd4d8_00007,ERROR,,0.259321,5,4,,50
train_hyper_model_fd4d8_00008,ERROR,,0.0758588,5,4,,100
train_hyper_model_fd4d8_00009,ERROR,,0.3447,5,2,,100


 pid=3580) 2022-02-11 19:04:26,672	ERROR worker.py:432 -- SystemExit was raised from the worker.
 pid=3580) Traceback (most recent call last):
 pid=3580)   File "python\ray\_raylet.pyx", line 629, in ray._raylet.execute_task
 pid=3580)   File "python\ray\_raylet.pyx", line 670, in ray._raylet.execute_task
 pid=3580)   File "python\ray\_raylet.pyx", line 636, in ray._raylet.execute_task
 pid=3580)   File "python\ray\_raylet.pyx", line 640, in ray._raylet.execute_task
 pid=3580)   File "python\ray\_raylet.pyx", line 589, in ray._raylet.execute_task.function_executor
 pid=3580)   File "c:\Users\huube\OneDrive\Master of Informatics\Machine Learning\Eindopdracht\.venv\lib\site-packages\ray\_private\function_manager.py", line 639, in actor_method_executor
 pid=3580)     return method(__ray_actor, *args, **kwargs)
 pid=3580)   File "c:\Users\huube\OneDrive\Master of Informatics\Machine Learning\Eindopdracht\.venv\lib\site-packages\ray\_private\function_manager.py", line 542, in temporary_acto

TuneError: ('Trials did not complete', [train_hyper_model_fd4d8_00000, train_hyper_model_fd4d8_00001, train_hyper_model_fd4d8_00002, train_hyper_model_fd4d8_00003, train_hyper_model_fd4d8_00004, train_hyper_model_fd4d8_00005, train_hyper_model_fd4d8_00006, train_hyper_model_fd4d8_00007, train_hyper_model_fd4d8_00008, train_hyper_model_fd4d8_00009, train_hyper_model_fd4d8_00010, train_hyper_model_fd4d8_00011, train_hyper_model_fd4d8_00012, train_hyper_model_fd4d8_00013, train_hyper_model_fd4d8_00014])